##### Table of Content

1. Intro
2. Solving a Sudoku
3. Setting up the Board
4. [Encoding the Board](#cell_encoding_the_board)
5. [Strategy1: Elimination](#cell_elimination)
6. [Strategy2: Only Choice](#cell_only_choice)
7. [Constraint Propagation](#cell_constraint_propagation)
8. [Harder Sudoku](#cell_harder_sudoku)
9. [Strategy3: Search](#cell_search)

** Goals of this project **

The main goal of this project is to build an intelligent agent that will solve every sudoku while introducing you to two powerful techniques that are used throughout the field of AI:

** Constraint Propagation ** 

When trying to solve a problem, you'll find that there are some local constraints to each square. These constraints help you narrow the possibilities for the answer, which can be very helpful. We will learn to extract the maximum information out of these constraints in order to get closer to our solution. Additionally, you'll see how we can repeatedly apply simple constraints to iteratively narrow the search space of possible solutions. Constraint propagation can be used to solve a variety of problems such as calendar scheduling, and cryptographic puzzles.

** Search **

In the process of problem solving, we may get to the point where two or more possibilities are available. What do we do? What if we branch out and consider both of them? Maybe one of them will lead us to a position in which three or more possibilities are available. Then, we can branch out again. At the end, we can create a whole tree of possibilities and find ways to traverse the tree until we find our solution. This is an example of how search can be used.
These ideas may seem simple and they're actually intended to be! Through this lesson you'll see how AI is really composed of very simple ideas that can be put together to solve complex problems. Throughout this lesson, we challenge you to think of how you can apply these ideas to build AI agents to solve other puzzles and problems in your world!

<a id="cell_encoding_the_board"></a>
## Step1: Encoding the Sudoku Board  

In [1]:
'''
Exercise1: Encoding the Board
>>> from utils import display
>>> display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . .
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return
    

#2. function.py ----------------------------
'''
Instruction: create grid_values(grid) A function to convert the string representation 
of a puzzle into a dictionary form.
'''
#2.1 grid_values function (input> string of sudoku problem, output> dictionary of a suduku with corresponding boxes) 
#from utils import *  #remove this when use as a file
def grid_values(grid):
    # In this function, you will take a sudoku as a string
    # and return a dictionary where the keys are the boxes,
    # for example 'A1', and the values are the digit at each
    # box (as a string) or '.' if the box has no value
    # assigned yet.
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    table = dict()
    index = 0
    for row_unit in row_units:
        for column in row_unit:
            table[column] = grid[index]
            index += 1
    print(table)
    print(type(table))
    return table

    
    
#3. Test function.py ----------------------------    
# from pprint import pprint
# pprint(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
# print(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

#4. Test utils.py ---------------------------- 
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
# print(units)

{'I6': '.', 'H9': '9', 'I2': '.', 'E8': '.', 'H3': '.', 'H7': '.', 'I7': '3', 'I4': '.', 'H5': '.', 'F9': '.', 'G7': '5', 'G6': '9', 'G5': '.', 'E1': '7', 'G3': '2', 'G2': '.', 'G1': '.', 'I1': '.', 'C8': '.', 'I3': '5', 'E5': '.', 'I5': '1', 'C9': '.', 'G9': '.', 'G8': '.', 'A1': '.', 'A3': '3', 'A2': '.', 'A5': '2', 'A4': '.', 'A7': '6', 'A6': '.', 'C3': '1', 'C2': '.', 'C1': '.', 'E6': '.', 'C7': '4', 'C6': '6', 'C5': '.', 'C4': '8', 'I9': '.', 'D8': '.', 'I8': '.', 'E4': '.', 'D9': '.', 'H8': '.', 'F6': '8', 'A9': '.', 'G4': '6', 'A8': '.', 'E7': '.', 'E3': '.', 'F1': '.', 'F2': '.', 'F3': '6', 'F4': '7', 'F5': '.', 'E2': '.', 'F7': '2', 'F8': '.', 'D2': '.', 'H1': '8', 'H6': '3', 'H2': '.', 'H4': '2', 'D3': '8', 'B4': '3', 'B5': '.', 'B6': '5', 'B7': '.', 'E9': '8', 'B1': '9', 'B2': '.', 'B3': '.', 'D6': '2', 'D7': '9', 'D4': '1', 'D5': '.', 'B8': '.', 'B9': '1', 'D1': '.'}
<type 'dict'>
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 .

<a id="cell_elimination"></a>
## Step2: Strategy1: Elimination
### <font color='red'> If a box has a value assigned, then none of the peers of this box can have this value. </font>

#### 2.1 First things first, let's look at a box and analyze the values that could go in there.
<img src="images/reduce-values-2.png" width="40%" height="40%"> 

#### 2.2 Elimination strategy
Now that we know how to eliminate values, we can take one pass, go over every box that has a value, and **eliminate the values that can't appear on the box, based on its peers**. Once we do so, the board looks like this 
<img src="images/values-easy.png" width="40%" height="40%">


<font color='red'>** This seems like something we can code!** </font>


In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/6rFOX2jHB2g" frameborder="0" allowfullscreen></iframe>

In [ ]:
'''
Exercise2.1: Improved grid_values()

As of now, we are recording the puzzles in dictionary form, where the keys are the boxes ('A1', 'A2', ... , 'I9') 
and the values are either the value for each box (if a value exists) or '.' (if the box has no value assigned yet). 
What we really want is for each value to represent all the available values for that box. 
For example, the box in the second row and fifth column above will have key 'B5' and value '47' 
(because 4 and 7 are the only possible values for it). The starting value for every empty box will thus be '123456789'.
Update the grid_values() function to return '123456789' instead of '.' for empty boxes.

>>> from utils import display
>>> display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3     123456789 123456789 
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return
    
#2. function.py ----------------------------
'''
Instruction : Update the grid_values() function to return '123456789' instead of '.' for empty boxes.
'''
# 2.1 improve grid_values(grid)
# from utils import *
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

#3. Test function.py ----------------------------  
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

In [2]:
'''
Exercise2.2: implement eliminate()

Now, let's finish the code for the function eliminate(), which will take as input a puzzle in dictionary form. 
The function will iterate over all the boxes in the puzzle that only have one value assigned to them, 
and it will remove this value from every one of its peers.

'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    table = dict()
    index = 0
    for row_unit in row_units:
        for column in row_unit:
            table[column] = grid[index]
            index += 1
    return table

#2. function.py ----------------------------
# 2.1 implement eliminate(values)
# from utils import *
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    eliminated_value = values
    for key, value in values.items():
        
        # ค่าตัดที่มีแล้วออก
        if value == '.':
            adjoins = units[key]
            adjoin_values = list()
            for adjoin in adjoins:
                # adjoin[0]
                # adjoin[1]
                # adjoin[2]
                for index in adjoin:
                    if values[index] != '.':
                        adjoin_values.append(values[index])
                        
            # หาค่าที่ยังไม่มี         
            remain_values = list()
            for num in range(1, 10, 1):
                num = str(num)
                if num not in adjoin_values:
                    remain_values.append(num)
            eliminated_value[key] = ''.join(remain_values)
            
    return eliminated_value

#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
eliminate(values)
print("\n")
print("After implement eliminate(values) method **********************************")
display(values)

The original Sudoku board is **********************************************
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


After implement eliminate(values) method **********************************
  45   4578   3   |  9     2    147  |  6    578    57  
  9    2678   7   |  3     4     5   |  78    28    1   
  25   257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1     36    2   |  9   34567 34567 
  7     23    9   |  5   34569   4   |  1   13456   8   
 1345  1345   6   |  7     39    8   |  2    345   345  
------------------+------------------+------------------
 134   1347   2   |  6     78    9   |  5    1478   47  
  8    147    47  |  2     57    3   |  17   1467   9   
  6    4679   5   |  4     1     47  |  3    

<a id="cell_only_choice"></a>
## Step3: Only Choice
### <font color='red'> Insight: Every unit must contain exactly one occurrence of every number </font>

#### 3.1 Look more carefully at the top 3x3 square in the center, highlighted in red.
<img src="images/highlighted-unit.png" width="40%" height="40%"> 

#### 3.2 Only Choice Strategy
<font color='red'> ** If there is only one box in a unit which would allow a certain digit, then that box must be assigned that digit. ** </font>

<img src="images/only-choice.png" width="40%" height="40%">

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/sSjYn-Kex1A" frameborder="0" allowfullscreen></iframe>

In [158]:
'''
Exercise3.1: implement only_choice()
Time to code it! In the next quiz, finish the code for the function only_choice, 
which will take as input a puzzle in dictionary form. The function will go through all the units, 
and if there is a unit with a digit that only fits in one possible box, it will assign that digit to that box.
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return


def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    table = dict()
    index = 0
    for row_unit in row_units:
        for column in row_unit:
            table[column] = grid[index]
            index += 1
    return table

#2. function.py ----------------------------
# 2.1 implement eliminate(values)
# from utils import *
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    eliminated_value = values
    for key, value in values.items():
        
        # ค่าตัดที่มีแล้วออก
        if value == '.':
            adjoins = units[key]
            adjoin_values = list()
            for adjoin in adjoins:
                for index in adjoin:
                    if values[index] != '.':
                        adjoin_values.append(values[index])
                        
            # หาค่าที่ยังไม่มี         
            remain_values = list()
            for num in range(1, 10, 1):
                num = str(num)
                if num not in adjoin_values:
                    remain_values.append(num)
            eliminated_value[key] = ''.join(remain_values)
            
    return eliminated_value

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    values = eliminate(values)
    
    for key, value in values.items():
        
        # หาค่าอื่นๆ ยังไม่แน่ใจ (มากกว่าสองตัว) ใน Box
        # ตัดช่องของตัวเองด้วย
            
#         if len(value) > 1:
#             box = units[key][2]
#             other_values = list()
#             for unit in box:
#                 if value == values[unit]:
#                     continue
#                 if len(unit) > 1:
#                     for digit in values[unit]:
#                         if digit not in other_values:
#                             other_values.append(digit)
                    
# #             print(value, other_values)
#             # หาค่าที่เหลือ => ค่าที่ไม่ซ้ำกับช่อง (box) อื่น
#             remainder = str()
#             for digit in value:
#                 if digit not in other_values:
#                     remainder += digit
# #             print(remainder)
#             if remainder:
#                 values[key] = remainder
# #                 print(values[key])

#             # เทียบกับอันที่มีหนึ่งตัวแล้วตัวออก
            
        # นับในกล่องว่าถ้ามี 1 ตัวให้ใช้ตัวนั้นไป
            if len(value) > 1:
                box = units[key][2]
                filtered_values = str()
                box_values = str()
                for unit in box:
                    for value in values[unit]:
                        if value not in box_values:
                            box_values += value
                print(box_values)
                    
            
                    
                
    return values

#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)
eliminate(values)
print("\n")
print("After implement eliminate(values) method **********************************")
display(values)

#4. Test function.py ----------------------------  
new_values = only_choice(values)
print("\n")
print("After implement only_choice(values) method **********************************")
display(new_values)

The original Sudoku board is **********************************************
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


After implement eliminate(values) method **********************************
   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1     356     8   
  1345  13459    6   |   7     359     8   |   2     345    345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  

In [3]:
'''
** Naked twins implementation understanding. **
In the README.md file, the student has shown an understanding of how constraint propagation
has been used to implement the naked twins function, by enforcing the constraint
that no squares outside the two naked twins squares can contain the twin values

** Diagonal Sudoku implementation understanding. **
In the README.md file, the student has shown an understanding of how constraint propagation
has been used to solve the diagonal sudoku, by adding the diagonals to the set of constraints.
'''

assignments = []

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s + t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in boxes)

# TODO please make use of this function
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s + t for s in A for t in B]

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print

def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            assign_value(values, peer, values[peer].replace(digit, ''))
    return values

def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                assign_value(values, dplaces[0], digit)
    return values


#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)
eliminate(values)
print("\n")
print("After implement eliminate(values) method **********************************")
display(values)

#4. Test function.py ----------------------------  
new_values = only_choice(values)
print("\n")
print("After implement only_choice(values) method **********************************")
display(new_values)



The original Sudoku board is **********************************************
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9

<a id="cell_constraint_propagation"></a>
## Step4: Constraint Propagation
### <font color='red'> using local constraints in a space to dramatically reduce the search space. </font>

#### 4.1 General explanation of constraint propagation
As we enforce each constraint, we see how it introduces new constraints for other parts of the board 
that can help us further reduce the number of possibilities. 

##### 4.1.1 Map Coloring Example 
##### <font color='red'> No two adjacent items can be the same color in the map coloring problem. </font>
<img src="images/map-coloring.jpg" width="40%" height="40%"> 

In the map coloring problem, we must find a way to color the map such that no two adjacent items share the same color. Indeed, we'll see how we use constraint propagation to use this simple constraint to find a solution just as we use such constraints to solve Sudoku.

##### 4.1.2 Crypto-Arithmetic Puzzles 
##### <font color='red'> what digits do T, W, O, F, U, and R represent? </font>
<img src="images/crypto_arithmetic_puzzle.png" width="40%" height="40%"> 

In Crypto-Arithmetic puzzles, each letter represents a digit, and no two letters represent the same digit. None of the numbers start with a leading zero. Our goal is to find a mapping from letters to digits that satisfies the equations. Here again, we'll find that the constraints imposed by the equation allow us to create an intelligent algorithm to solve the problem via Constraint Propagation.

#### 4.2 Applying Constraint Propagation to Sudoku
<font color='red'> ** combine the functions eliminate() and only_choice() to write the function reduce_puzzle(), which receives as input an unsolved puzzle and applies our two constraints repeatedly in an attempt to solve it. ** </font>

** Some things to watch out for: **

- The function needs to stop if the puzzle gets solved. How to do this?
- What if the function doesn't solve the sudoku? Can we make sure the function quits when applying the two strategies stops making progress?

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/aSYDBcbvC5Y" frameborder="0" allowfullscreen></iframe>

In [180]:
'''
Exercise4.1: Apply Constraint Propagation to Sudoku problem
Now that you see how we apply Constraint Propagation to this problem, let's try to code it! 
In the following quiz, combine the functions eliminate and only_choice to write the function reduce_puzzle, 
which receives as input an unsolved puzzle and applies our two constraints repeatedly in an attempt to solve it.

Some things to watch out for:
- The function needs to stop if the puzzle gets solved. How to do this?
- What if the function doesn't solve the sudoku? Can we make sure the function quits when applying 
the two strategies stops making progress?
'''

#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """

    table = dict()
    index = 0
    for row_unit in row_units:
        for column in row_unit:
            table[column] = grid[index]
            index += 1
    return table

#2. function.py ----------------------------
# 2.1 implement eliminate(values)
# from utils import *
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    eliminated_value = values
    for key, value in values.items():
        
        # กำหนดค่าให้แทน '.'
        if value == '.':
            adjoins = units[key]
            adjoin_values = list()
            for adjoin in adjoins:
                for index in adjoin:
                    if len(values[index]) == 1:
                        adjoin_values.append(values[index])
                        
            # หาค่าที่ยังไม่มี         
            remain_values = list()
            for num in range(1, 10, 1):
                num = str(num)
                if num not in adjoin_values:
                    remain_values.append(num)
            eliminated_value[key] = ''.join(remain_values)
            
        # กำจัดค่าที่ซ้ำ โดยเช็คแกน X, Y และใน Box
        elif len(value) > 1:
            adjoins = units[key]
            adjoin_values = list()
            for adjoin in adjoins:
                for index in adjoin:
                    if len(values[index]) == 1 and values[index] not in adjoin_values:
                        adjoin_values.append(values[index])
                        
            # เอาค่าที่ยังไม่มีใน Box มาใส่ใน Unit         
            remain_values = list()
            for num in range(1, 10, 1):
                num = str(num)
                if num not in adjoin_values:
                    remain_values.append(num)
            eliminated_value[key] = ''.join(remain_values)
            
    return eliminated_value

# 2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    for key, value in values.items():
        
        # หาค่าอื่นๆ ยังไม่แน่ใจ (มากกว่าสองตัว) ใน Box
        # ตัดช่องของตัวเองด้วย
        if len(value) > 1:
            box = units[key][2]
            
            # เช็คซ้ำกับช่องอื่นใน Box ก่อน
            # ถ้ามีซ้ำ ให้ continue
            for unit in box:
                if value == values[unit]:
                    continue
            
            # ถ้ามีไม่ซ้ำ ให้หาค่าที่มีใน Box อยู่แล้วมาใส่ other_values
            other_values = list()
            for unit in box:
                if len(unit) > 1:
                    for digit in values[unit]:
                        if digit not in other_values:
                            other_values.append(digit)
                    
            # หาค่าที่เหลือ => ค่าที่ไม่ซ้ำกับช่อง (box) อื่น
            remainder = str()
            for digit in value:
                if digit not in other_values:
                    remainder += digit
            if remainder:
                values[key] = remainder
                
    return values

#2. function.py ----------------------------
# 2.1 combine the functions eliminate and only_choice to write the function reduce_puzzle
# from utils import *
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    stalled = False
    old_values = dict()
    round_count = 0
    while not stalled:
        round_count += 1
        if old_values == values:
            break
        else:
            # ให้ old_values = values เพื่อเปรียบเทียบในรอบต่อไป
            old_values = values.copy()
            
        values = eliminate(values)
#         print('round count:', round_count)
#         display(values)
        values = only_choice(values)
    
    return values

#3. Test utils.py ----------------------------  
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = reduce_puzzle(values)
print("\n")
print("After applying constrint propagaton (both eliminate and only_choice strategies)*****************")
display(new_values)

The original Sudoku board is **********************************************
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


After applying constrint propagaton (both eliminate and only_choice strategies)*****************
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


##  <font color='red'> So, that seemed to work! Congratulation You should have got this answer. </font>
<img src="images/easy-solution.png" width="40%" height="40%"> 

<a id="cell_harder_sudoku"></a>
## Step5: Harder Sudoku
### <font color='red'> Ok, let's see if our algorithm will work all the time. Here's a harder sudoku puzzle </font>

<img src="images/harder-puzzle.png" width="40%" height="40%"> 

In [ ]:
'''
Exercise5.1: Try to solve harder sudoku using the same constraint propagation
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

#2. function.py ----------------------------
# 2.1 combine the functions eliminate and only_choice to write the function reduce_puzzle
# from utils import *
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

#3. Test utils.py ----------------------------  
grid_easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid_hard)
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = reduce_puzzle(values)
print("\n")
print("After applying constrint propagaton (both eliminate and only_choice strategies)*****************")
display(new_values)

##  <font color='red'> Oh no! The algorithm didn't solve it. It seemed to reduce every box to a number of possibilites, but it won't go farther than that. We need to think of other ways to improve our solution. </font>
<img src="images/harder-sudoku-reduced.png" width="40%" height="40%"> 

<a id="cell_search"></a>
## Step6: Strategy 3: Search
### <font color='red'> Search is used throughout AI from Game-Playing to Route Planning to efficiently find solutions. </font>

#### 6.1 An example of Search being used in Google's AlphaGo paper.
Learn more about Google's AlphaGo paper here: [Mastering the game of Go with deep neural networks and tree search](https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf) 
<img src="images/search_alpha_go.png" width="70%" height="70%"> 

#### 6.2 Search Strategy
##### <font color='red'> Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively. </font>

Before we dive in to code the search function, let's first check our understanding. How would you traverse the following tree using Depth First Search?

<img src="images/bfs-quiz.png" width="40%" height="40%"> 

** DFS Quiz ** 
Traverse the above tree using Depth First Search. The answer should be the string obtained by the labels in the order you've traversed the tree. For example, if your tree has four vertices, A, B, C, D, and you've traversed them in the order B->C->A->D, then the answer should be the string 'BCAD'.

** DFS Answer ** >> 'ABDEHIJCFGKL'

##### <font color='red'> In our example, the box 'G2' has two possibilities: 8 and 9. Why don't we fill it in with a 8 and try to solve our puzzle. </font>
<img src="images/choices.png" width="40%" height="40%"> 

In [ ]:
%%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/BzwTKFbpnXE" frameborder="0" allowfullscreen></iframe>

In [ ]:
'''
Exercise6.1: Coding the Solution

Time to code the final solution! Finish the code in the function search, 
which will create a tree of possibilities and traverse it using DFS until it finds a solution for the sudoku puzzle.
'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''
    
#2. function.py ----------------------------
# 2.1 implement search() using Depth First Search Algorithm
#from utils import *
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    # Search and Choose one of the unfilled squares with the fewest possibilities
    # Now use recursion to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!
    
    ''' Your solution here 
    .........
    .........
    .........
    .........
    .........
    .........
    '''

#3. Test utils.py ----------------------------  
grid_easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid_hard)
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = search(values)
print("\n")
print("After applying Depth First Search Algorithm *****************")
display(new_values)

##  <font color='red'> So, that seemed to work! Congratulation You should have got this answer. </font>
<img src="images/hard-solution.png" width="40%" height="40%"> 